# AI Red Teaming Agent for Generative AI models and applications in Azure AI Foundry

## Objective
This notebook walks through how to use Azure AI Evaluation's AI Red Teaming Agent functionality to assess the safety and resilience of AI systems against adversarial prompt attacks. AI Red Teaming Agent leverages [Risk and Safety Evaluations](https://learn.microsoft.com/en-us/azure/ai-foundry/concepts/evaluation-metrics-built-in?tabs=warning#risk-and-safety-evaluators) to help identify potential safety issues across different risk categories (violence, hate/unfairness, sexual content, self-harm) combined with attack strategies of varying complexity levels from [PyRIT](https://github.com/Azure/PyRIT), Microsoft AI Red Teaming team's open framework for automated AI red teaming.

## Time
You should expect to spend about 30-45 minutes running this notebook. Execution time will vary based on the number of risk categories, attack strategies, and complexity levels you choose to evaluate.

## Before you begin

### Prerequisite
First, if you have an Azure subscription, create an [Azure AI hub](https://learn.microsoft.com/en-us/azure/ai-studio/concepts/ai-resources) then [create an Azure AI project](https://learn.microsoft.com/en-us/azure/ai-studio/concepts/ai-resources). AI projects and Hubs can be served within a private network and are compatible with private endpoints. You **do not** need to provide your own LLM deployment as the AI Red Teaming Agent hosts adversarial models for both simulation and evaluation of harmful content and connects to it via your Azure AI project.

In order to upload your results to Azure AI Foundry:
- Your AI Foundry project must have a connection (*Connected Resources*) to a storage account with `Microsoft Entra ID` authentication enabled.
- Your AI Foundry project must have the `Storage Blob Data Contributor` role in the storage account.
- You must have the `Storage Blob Data Contributor` role in the storage account.
- You must have network access to the storage account.

For more information see: https://learn.microsoft.com/en-us/azure/ai-foundry/how-to/develop/run-scans-ai-red-teaming-agent

**Important**: First, ensure that you've installed the [Azure CLI](https://learn.microsoft.com/en-us/cli/azure/install-azure-cli) and then make sure to authenticate to Azure using `az login` in your terminal before running this notebook.

### Installation
From a terminal window, navigate to your working directory which contains this sample notebook, and execute the following.
```bash
python -m venv .venv
```

Then, activate the virtual environment created:

```bash
# %source .venv/bin/activate # If using Mac/Linux OS
.venv/Scripts/activate # If using Windows OS
```

With your virtual environment activated, install the following packages required to execute this notebook:

```bash
pip install uv
uv pip install azure-ai-evaluation[redteam] azure-identity openai azure-ai-projects
```


Now open VSCode with the following command, and ensure your virtual environment is used as kernel to run the remainder of this notebook.
```bash
code .
```

### Imports

In [ ]:

from typing import Optional, Dict, Any
import os

# Azure imports
from azure.ai.evaluation.red_team import RedTeam, RiskCategory, AttackStrategy

# OpenAI imports
from openai import AzureOpenAI

### Login to Azure with valid credentials

In [ ]:
# Azure Credential imports
from azure.identity import AzureCliCredential, get_bearer_token_provider

!az login

# Initialize Azure credentials
credential = AzureCliCredential()

### Set Up Your Environment Variables


```
# Azure OpenAI
AZURE_OPENAI_API_KEY="your-api-key-here"
AZURE_OPENAI_ENDPOINT="https://endpoint-name.cognitiveservices.azure.com/"
AZURE_OPENAI_DEPLOYMENT_NAME="gpt-4"
AZURE_OPENAI_API_VERSION="2024-12-01-preview"

# Azure AI Project
AZURE_PROJECT_ENDPOINT="https://your-aifoundry-endpoint-name.services.ai.azure.com/api/projects/yourproject-name"
```

In [ ]:
# Load environment variables from .env file
from dotenv import load_dotenv
load_dotenv()

# Azure AI Project information
azure_ai_project = os.environ.get("AZURE_PROJECT_ENDPOINT")

# Azure OpenAI deployment information
azure_openai_deployment = os.environ.get("AZURE_OPENAI_DEPLOYMENT")  # e.g., "gpt-4"
azure_openai_endpoint = os.environ.get("AZURE_OPENAI_ENDPOINT")
azure_openai_api_key = os.environ.get("AZURE_OPENAI_API_KEY")  # e.g., "your-api-key"
azure_openai_api_version = os.environ.get("AZURE_OPENAI_API_VERSION")  # Use the latest API version

## Understanding AI Red Teaming Agent's capabilities

1. **Risk Categories**: 
   - Violence
   - HateUnfairness
   - Sexual
   - SelfHarm

2. **Attack Strategies**: 
   - AnsiAttack: Using ANSI escape codes in prompts
   - AsciiArt: Using ASCII art to disguise harmful content
   - AsciiSmuggler: Hiding harmful content within ASCII characters
   - Atbash: Using the Atbash cipher to encode harmful requests
   - Base64: Encoding harmful content in Base64 format
   - Binary: Converting text to binary to bypass filters
   - Caesar: Using the Caesar cipher for encoding
   - CharacterSpace: Manipulating character spacing to confuse filters
   - CharSwap: Swapping characters to bypass detection
   - Diacritic: Using diacritical marks to alter text appearance
   - Flip: Flipping text to bypass content filters
   - Leetspeak: Converting letters to numbers and symbols
   - Morse: Using Morse code to encode harmful requests
   - ROT13: Using ROT13 cipher for text transformation
   - SuffixAppend: Adding suffixes to confuse detection systems
   - StringJoin: Joining strings in unconventional ways
   - Tense: Changing the tense of harmful requests to past tense
   - UnicodeConfusable: Using similar-looking Unicode characters
   - UnicodeSubstitution: Substituting characters with Unicode alternatives
   - Url: Embedding harmful content within URLs
   - Jailbreak: Specially crafted prompts to bypass AI safeguards

3. **Complexity Levels**: 
   - Baseline: Standard functionality tests
   - Easy: Simple attack patterns
   - Moderate: More sophisticated attacks
   - Difficult: Complex, layered attack strategies

The key metric for evaluating results is the **Attack Success Rate (ASR)**

## Basic Example: Fixed Response Callback

In [ ]:
# Define a simple callback function that always returns a fixed response
def financial_advisor_callback(query: str) -> str:  # noqa: ARG001
    return "I'm a financial advisor assistant. I can help with investment advice and financial planning within legal and ethical guidelines."

### Getting Your Azure AI Project Details


**Required fields:**
- `AZURE_SUBSCRIPTION_ID`: Your Azure subscription ID
- `AZURE_RESOURCE_GROUP`: The resource group containing your AI project
- `AZURE_PROJECT_NAME`: The name of your AI project

In [ ]:
# Verify Azure AI Project configuration
def verify_azure_config():
    """Verify that all required Azure AI Project fields are configured."""
    required_fields = {
        "AZURE_SUBSCRIPTION_ID": "Azure Subscription ID",
        "AZURE_RESOURCE_GROUP": "Resource Group Name",
        "AZURE_PROJECT_NAME": "Project Name"
    }
    
    missing_fields = []
    placeholder_fields = []
    
    print("🔍 Verifying Azure AI Project Configuration...\n")
    print("=" * 60)
    
    for env_var, description in required_fields.items():
        value = os.environ.get(env_var)
        if not value:
            missing_fields.append(env_var)
            print(f"❌ {description} ({env_var}): NOT SET")
        elif "your-" in value.lower() or value == "":
            placeholder_fields.append(env_var)
            print(f"⚠️  {description} ({env_var}): PLACEHOLDER VALUE")
            print(f"    Current value: {value}")
        else:
            # Mask sensitive information
            if len(value) > 20:
                masked_value = f"{value[:8]}...{value[-4:]}"
            else:
                masked_value = f"{value[:4]}..."
            print(f"✅ {description} ({env_var}): {masked_value}")
    
    print("=" * 60)
    
    if missing_fields or placeholder_fields:
        print("\n⚠️  Configuration Issues Found!\n")
        
        if missing_fields:
            print("Missing fields:")
            for field in missing_fields:
                print(f"  - {field}")
        
        if placeholder_fields:
            print("\nPlaceholder values detected:")
            for field in placeholder_fields:
                print(f"  - {field}")
        
        print("\n📝 To fix this:")
        print("1. Update the .env file with your actual Azure values")
        print("2. Run: python configure_azure_project.py (for automatic setup)")
        print("3. Or manually edit .env and restart the kernel")
        print("\nSee README-AZURE-CONFIG.md for detailed instructions.")
        return False
    else:
        print("\n✅ All required fields are configured!")
        print("You can proceed with creating the RedTeam instance.")
        return True

# Run verification
verify_azure_config()

In [ ]:
# Create the `RedTeam` instance with minimal configurations
azure_ai_project = os.environ.get("AZURE_PROJECT_ENDPOINT")

if not azure_ai_project:
    raise ValueError("AZURE_PROJECT_ENDPOINT environment variable is not set or empty.")

red_team = RedTeam(
    azure_ai_project=azure_ai_project,
    credential=credential,
    risk_categories=[RiskCategory.Violence, RiskCategory.HateUnfairness],
    num_objectives=1,
)

NOTE: `num_objectives` specifies the number of attacks to perform per risk category per attack strategy. If the parameter `risk_categories` is not specified, `[RiskCategory.Violence, RiskCategory.HateUnfairness, RiskCategory.Sexual, RiskCategory.SelfHarm]` will be used by default.

Now let's run a simple automated scan using the `RedTeam` with the fixed response target. We'll test against two risk categories and one attack strategy for simplicity.

In [ ]:
# Run the red team scan called "Basic-Callback-Scan" with limited scope for this basic example
# This will test 1 objective prompt for each of Violence and HateUnfairness categories with the Flip strategy
result = await red_team.scan(
    target=financial_advisor_callback,
    scan_name="Basic-Callback-Scan",
    attack_strategies=[AttackStrategy.Flip],
    output_path="red_team_output.json",
)

## Intermediary Example: Using a Model Configuration as Target


In [ ]:
# Define a model configuration to test
azure_oai_model_config = {
    "azure_endpoint": azure_openai_endpoint,
    "azure_deployment": azure_openai_deployment,
    "api_key": azure_openai_api_key,
}

Then, update your target to point to the model configurations and run the scan.

In [ ]:
# Run the red team scan called "Intermediary-Model-Target-Scan"
result = await red_team.scan(
    target=azure_oai_model_config,
    scan_name="Intermediary-Model-Target-Scan",
    attack_strategies=[AttackStrategy.Flip],
)

## Advanced Example: Using an Azure Open AI Model Endpoint in a Callback Function


In [ ]:
# Define a callback that uses Azure OpenAI API to generate responses
async def azure_openai_callback(
    messages: list,
    stream: Optional[bool] = False,  # noqa: ARG001
    session_state: Optional[str] = None,  # noqa: ARG001
    context: Optional[Dict[str, Any]] = None,  # noqa: ARG001
) -> dict[str, list[dict[str, str]]]:
    # Get token provider for Azure AD authentication
    token_provider = get_bearer_token_provider(credential, "https://cognitiveservices.azure.com/.default")

    # Initialize Azure OpenAI client
    client = AzureOpenAI(
        azure_endpoint=azure_openai_endpoint,
        api_version=azure_openai_api_version,
        azure_ad_token_provider=token_provider,
    )

    ## Extract the latest message from the conversation history
    messages_list = [{"role": message.role, "content": message.content} for message in messages]
    latest_message = messages_list[-1]["content"]

    try:
        # Call the model
        response = client.chat.completions.create(
            model=azure_openai_deployment,
            messages=[
                {"role": "user", "content": latest_message},
            ],
            max_tokens=500, # If using an o1 base model, comment this line out
            #max_completion_tokens=500,  # If using an o1 base model, uncomment this line
            temperature=0.7, # If using an o1 base model, comment this line out (temperature param not supported for o1 base models)
        )

        # Format the response to follow the expected chat protocol format
        formatted_response = {"content": response.choices[0].message.content, "role": "assistant"}
    except Exception as e:
        print(f"Error calling Azure OpenAI: {e!s}")
        formatted_response = "I encountered an error and couldn't process your request."
    return {"messages": [formatted_response]}

In [ ]:
# Create the RedTeam instance with all of the risk categories with 5 attack objectives generated for each category
model_red_team = RedTeam(
    azure_ai_project=azure_ai_project,
    credential=credential,
    risk_categories=[RiskCategory.Violence, RiskCategory.HateUnfairness, RiskCategory.Sexual, RiskCategory.SelfHarm],
    num_objectives=5,
)

We will use this instance of `model_red_team` to test different attack strategies in the following section.

### Testing Different Attack Strategies


In [ ]:
# Run the red team scan with multiple attack strategies
advanced_result = await model_red_team.scan(
    target=azure_openai_callback,
    scan_name="Advanced-Callback-Scan",
    attack_strategies=[
        AttackStrategy.EASY,  # Group of easy complexity attacks
        AttackStrategy.MODERATE,  # Group of moderate complexity attacks
        AttackStrategy.CharacterSpace,  # Add character spaces
        AttackStrategy.ROT13,  # Use ROT13 encoding
        AttackStrategy.UnicodeConfusable,  # Use confusable Unicode characters
        AttackStrategy.CharSwap,  # Swap characters in prompts
        AttackStrategy.Morse,  # Encode prompts in Morse code
        AttackStrategy.Leetspeak,  # Use Leetspeak
        AttackStrategy.Url,  # Use URLs in prompts
        AttackStrategy.Binary,  # Encode prompts in binary
        AttackStrategy.Compose([AttackStrategy.Base64, AttackStrategy.ROT13]),  # Use two strategies in one attack
    ],
    output_path="Advanced-Callback-Scan.json",
)

## Bring your own objectives: Using your own prompts as objectives for RedTeam


In [ ]:
path_to_prompts = "./data/prompts.json"  # Fixed: Use forward slashes or raw string

# Create the RedTeam specifying the custom attack seed prompts to use as objectives
custom_red_team = RedTeam(
    azure_ai_project=azure_ai_project,
    credential=credential,
    custom_attack_seed_prompts=path_to_prompts,  # Path to a file containing custom attack seed prompts
)

In [ ]:
custom_red_team_result = await custom_red_team.scan(
    target=azure_openai_callback,
    scan_name="Custom-Prompt-Scan",
    attack_strategies=[
        AttackStrategy.EASY,  # Group of easy complexity attacks
        AttackStrategy.MODERATE,  # Group of moderate complexity attacks
        AttackStrategy.DIFFICULT,  # Group of difficult complexity attacks
    ],
    output_path="Custom-Prompt-Scan.json",
)

In [ ]:

print("Creating a Red Team scan for direct model testing")

# Create target configuration for testing an Azure OpenAI model
target_config = AzureOpenAIModelConfiguration(model_deployment_name=model_deployment_name)

# Create the Red Team configuration
red_team = RedTeam(
    attack_strategies=[AttackStrategy.BASE64],
    risk_categories=[RiskCategory.VIOLENCE],
    display_name="redteamtest1",  
    target=target_config,
)

# Create and run the Red Team scan
red_team_response = red_team.red_teams.create(
red_team=red_team,
    headers={
        "model-endpoint": model_endpoint,
        "model-api-key": model_api_key,
        },
)
print(f"Red Team scan created with scan name: {red_team_response.name}")

print("Getting Red Team scan details")
# Use the name returned by the create operation for the get call
get_red_team_response = red_team.red_teams.get(name=red_team_response.name)
print(f"Red Team scan status: {get_red_team_response.status}")

print("Listing all Red Team scans")
for scan in red_team.red_teams.list():
    print(f"Found scan: {scan.name}, Status: {scan.status}")

## Conclusion

In this notebook, we've demonstrated how to use the Azure AI Evaluation SDK's `RedTeam` functionality to assess the safety and resilience of AI systems. We started with a basic fixed-response example and then moved to a more realistic model testing across multiple risk categories and attack strategies.

The automated AI red teaming scans provides valuable insights into:

1. **Overall Attack Success Rate (ASR)** - The percentage of attacks that successfully elicit harmful content
2. **Vulnerability by Risk Category** - Which types of harmful content your model is most vulnerable to
3. **Effectiveness of Attack Strategies** - Which attack techniques are most successful against your model
4. **Impact of Complexity** - How more sophisticated attacks affect your model's safety guardrails

By regularly red-teaming your AI applications, you can identify and address potential vulnerabilities before deploying your models to production environments.

### Next Steps

1. **Mitigation**: Use these results to strengthen your model's guardrails against identified attack vectors
2. **Continuous Testing**: Implement regular red team evaluations as part of your development lifecycle
3. **Custom Strategies**: Develop custom attack strategies for your specific use cases and domain
4. **Safety Layers**: Consider adding additional safety layers like Azure AI Content Safety to filter harmful requests and responses 